In [ ]:
import ast
import re

# Visitor to collect names (variables, functions) for snake_case and camelCase convention checking
class NameConventionVisitor(ast.NodeVisitor):
    def __init__(self):
        self.all_names = set()     # All names (variables, functions, etc.)
        self.snake_case_names = set()
        self.camel_case_names = set()
        self.other_case_names = set()

    def visit_FunctionDef(self, node):
        self.all_names.add(node.name)
        self._check_name_convention(node.name)
        self.generic_visit(node)

    def visit_Name(self, node):
        # Collect all variable names
        self.all_names.add(node.id)
        self._check_name_convention(node.id)
        self.generic_visit(node)

    def _check_name_convention(self, name):
        snake_case_pattern = r'^[a-z]+(_[a-z0-9]+)*$'
        camel_case_pattern = r'^[a-z]+([A-Z][a-z0-9]+)+$'

        if re.match(snake_case_pattern, name):
            self.snake_case_names.add(name)
        elif re.match(camel_case_pattern, name):
            self.camel_case_names.add(name)
        else:
            self.other_case_names.add(name)

# Checker to ensure all names follow the expected convention (default: snake_case)
class NameConventionChecker:
    def __init__(self):
        self.violations = set()

    def check_conventions(self, snake_case_names, camel_case_names, other_case_names):
        # Check names in camelCase or other formats that should follow snake_case
        for name in camel_case_names:
            self.violations.add(f"Violation: '{name}' is in camelCase, expected snake_case.")
        for name in other_case_names:
            self.violations.add(f"Violation: '{name}' does not follow snake_case or camelCase.")

# Sample code to test NameConventionVisitor and NameConventionChecker
code = """
def exampleFunction():
    snake_case_var = 10
    camelCaseVar = 20
    AnotherVar = 30
"""

# Parse the code
tree = ast.parse(code)

# Run NameConventionVisitor
name_visitor = NameConventionVisitor()
name_visitor.visit(tree)

# Run NameConventionChecker
name_checker = NameConventionChecker()
name_checker.check_conventions(name_visitor.snake_case_names, 
                               name_visitor.camel_case_names, 
                               name_visitor.other_case_names)

# Print the name convention violations
print("Name convention violations:", name_checker.violations)
